In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import sklearn
import os
import time
import sys
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)

for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.2
numpy 1.17.4
pandas 0.25.3
sklearn 0.22
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()

x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(55000, 28, 28) (55000,)
(5000, 28, 28) (5000,)
(10000, 28, 28) (10000,)


In [3]:
# x = (x-u) / std

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# x_train: [None, 28, 28] -> [None, 784]
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)

In [4]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# reason for sparse: y is index use sparse, y is one_hot vector use categorical_crossentropy
model.compile(loss='sparse_categorical_crossentropy',
             optimiser='sgd',
             metrics=['accuracy'])

In [5]:
history = model.fit(x_train_scaled, y_train, epochs=10,
                   validation_data=(x_valid, y_valid))

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 12s 221us/sample - loss: 0.4884 - accuracy: 0.8247 - val_loss: 66.3210 - val_accuracy: 0.7600
Epoch 2/10
55000/55000 [==============================] - 11s 201us/sample - loss: 0.3893 - accuracy: 0.8635 - val_loss: 114.9574 - val_accuracy: 0.7896
Epoch 3/10
55000/55000 [==============================] - 11s 207us/sample - loss: 0.3685 - accuracy: 0.8730 - val_loss: 234.7619 - val_accuracy: 0.7844
Epoch 4/10
55000/55000 [==============================] - 12s 212us/sample - loss: 0.3542 - accuracy: 0.8803 - val_loss: 236.4366 - val_accuracy: 0.8000
Epoch 5/10
55000/55000 [==============================] - 11s 202us/sample - loss: 0.3384 - accuracy: 0.8855 - val_loss: 494.9902 - val_accuracy: 0.7306
Epoch 6/10
55000/55000 [==============================] - 11s 194us/sample - loss: 0.3282 - accuracy: 0.8887 - val_loss: 297.6026 - val_accuracy: 0.8266
Epoch 7/10
55000/55000 [==========

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 3)
    plt.show()

plot_learning_curves(history)

In [6]:
model.evaluate(x_test_scaled, y_test)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.611008746920526, 0.8666]

In [10]:
tf.saved_model.save(model, './keras_saved_graph')

INFO:tensorflow:Assets written to: ./keras_saved_graph/assets


In [9]:
!saved_model_cli show --dir ./keras_saved_graph --all

/bin/sh: saved_model_cli: command not found


In [ ]:
!saved_model_cli show --dir ./keras_saved_graph --tag_set serve --signature_def serving_default

In [ ]:
!saved_model_cli show --dir ./keras_saved_graph --tag_set serve
    --signature_def serving_default
    --input_exprs 'flatten_input=np.ones((2, 28, 28))'

In [11]:
loaded_saved_model = tf.saved_model.load('./keras_saved_graph')
print(loaded_saved_model.signatures.keys())

KeysView(_SignatureMap({'serving_default': <tensorflow.python.saved_model.load._WrapperFunction object at 0x1422fe350>}))


In [12]:
inference = loaded_saved_model.signatures['serving_default']
print(inference)

In [13]:
print(inference.structured_outputs)

{'dense_2': TensorSpec(shape=(None, 10), dtype=tf.float32, name='dense_2')}


In [17]:
results = inference(tf.constant(x_test_scaled[0:1]))
print(results['dense_2'])

tf.Tensor(
[[1.1070039e-15 3.7269834e-18 5.2455630e-22 2.3759987e-13 9.9215692e-23
  6.7155787e-07 4.4576039e-18 8.7685343e-09 9.2246107e-12 9.9999928e-01]], shape=(1, 10), dtype=float32)
